# Inference
Now that we have our finetuned model, we are going to generate the triples from the test text.

## Import libraries and LLM

In [ ]:
!pip install matplotlib
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 24.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=False,
    add_eos_token=False,
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Load Test Data

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
test_df = pd.read_csv("test.csv")["0"]

In [ ]:
test_df = list(test_df)

In [ ]:
formatted_test_df = []

In [ ]:
# For the base model, we are going to add this sentence in order to give it instructions on how to format the triples
sentence = """Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)" """
sentence = sentence.strip()

In [ ]:
test_df[0]

'<s>[INST]Imagine being a Knowledge Graph constructor from unstructured text.\nFollowing the schema provided, extract all the triples you can find in the text.\n\nSchema:\n(Legislator)-[REPRESENTS]->(StateCode)\n(Legislator)-[IS_MEMBER_OF]->(Party)\n(Legislator)-[ELECTED_TO]->(Body)\n(BillId)-[SPONSORED_BY]->(Legislator)\n(Legislator)-[VOTED_ON]->(BillId)\n(BillId)-[REFERRED_TO]->(Committee)\n(Legislator)-[SERVES_ON]->(Committee)\n(BillId)-[DEALS_WITH]->(Subject)\n\nContext:\n"Hi and welcome to our show about very interesting bills.\nToday\'s bill is the hr1818-114 bill. This bill was proposed by Ken Calvert, a Republican representative from California, and Grace Napolitano, a representative from California who is elected to the House. These two representatives are from the same state and share the sponsorship of the same bill."[/INST]\n\nExtracted Triples:\n(Grace Napolitano) - [REPRESENTS] -> (CA)\n(Grace Napolitano) - [ELECTED_TO] -> (House)\n(Ken Calvert) - [REPRESENTS] -> (CA)\n(K

In [ ]:
for el in tqdm(test_df):
    new_diz = {}
    splitted_el = el.split("[/INST]\n\nExtracted Triples:")
    text = splitted_el[0][3:].strip()
    text = text.split("Following the schema provided, extract all the triples you can find in the text.")
    text = sentence.join(text)
    text += "[/INST]\n\nExtracted Triples:"
    new_diz["text"] = text
    new_diz["triples"] = splitted_el[1][:-4].strip()
    formatted_test_df.append(new_diz)

100%|██████████| 400/400 [00:00<00:00, 182143.26it/s]


In [ ]:
formatted_test_df[0]

{'text': '[INST]Imagine being a Knowledge Graph constructor from unstructured text.\nFollowing the schema provided, extract all the triples you can find in the text.\nMake sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n\nSchema:\n(Legislator)-[REPRESENTS]->(StateCode)\n(Legislator)-[IS_MEMBER_OF]->(Party)\n(Legislator)-[ELECTED_TO]->(Body)\n(BillId)-[SPONSORED_BY]->(Legislator)\n(Legislator)-[VOTED_ON]->(BillId)\n(BillId)-[REFERRED_TO]->(Committee)\n(Legislator)-[SERVES_ON]->(Committee)\n(BillId)-[DEALS_WITH]->(Subject)\n\nContext:\n"Hi and welcome to our show about very interesting bills.\nToday\'s bill is the hr1818-114 bill. This bill was proposed by Ken Calvert, a Republican representative from California, and Grace Napolitano, a representative from California who is elected to the House. These two representatives are from the same state and share the sponsorship of the same bill."[/INST]\n\nExtracted Triples:',
 'triples': '(Grace Napolitano) - [R

# Generate responses (base model)
Generate the triples starting from the test set for Mistral 7B Instruct non-finetuned

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left")

base_model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    response = generate_response(el["text"].strip())
    el["NF_triples"] = response

100%|██████████| 400/400 [1:05:02<00:00,  9.76s/it]


In [ ]:
print(formatted_test_df[2]["NF_triples"])

<s> [INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)

Context:
"**Higher Education Enthusiasts Rejoice!**
Calling all avid learners, knowledge seekers, and future leaders! Prepare yourselves for a tantalizing treat as we dive into the world of higher education.
From the esteemed halls of hr2133-114, we unravel the complexities of both vocational and technical education. But that's not all! Also hailing from hr2133-114, brace yourselves for a journey through the realm of higher education.
Embarking on a sim

In [ ]:
for el in formatted_test_df:
    el["NF_triples"] = el["NF_triples"].split("Extracted Triples:")[-1].strip()

In [ ]:
print(formatted_test_df[2]["NF_triples"])

* (hr2133-114)-[REPRESENTS]->(vocational and technical education)
* (hr2133-114)-[SERVES_ON]->(committee)
* (hr5310-114)-[REPRESENTS]->(higher education)
* (hr3206-114)-[REPRESENTS]->(higher education)
* (hr4371-114)-[REPRESENTS]->(higher education)
* (hr386-114)-[REPRESENTS]->(higher education)
* (hr4803-114)-[REPRESENTS]->(educational pursuits)
* (hr3180-114)-[REPRESENTS]->(educational pursuits)
* (hr4098-114)-[REPRESENTS]->(pursuing knowledge beyond the ordinary)</s>


In [ ]:
# Save the responses
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('NFresponses.json', 'w') as f:
    json.dump(json_str, f)

# Generate responses (base model with few shot prompting)
Generate the triples starting from the test set for Mistral 7B Instruct non-finetuned with 5 examples.

In [ ]:
# importing the module
import json

# Opening JSON file
with open('NFresponses.json') as json_file:
    data = json.load(json_file)

In [ ]:
formatted_test_df = json.loads(data)

In [ ]:
print(formatted_test_df[0]["text"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)

Context:
"Hi and welcome to our show about very interesting bills.
Today's bill is the hr1818-114 bill. This bill was proposed by Ken Calvert, a Republican representative from California, and Grace Napolitano, a representative from California who is elected to the House. These two representatives are from the same state and share the sponsorship of the same bill."[/INST]

Extracted Triples:


## Build examples

In [ ]:
from datasets import load_dataset
# Load Dataset
data = load_dataset("Giardooo/KG_constructor")
print(data)

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['0'],
        num_rows: 400
    })
})


In [ ]:
print(data["train"][1]["0"])

<s>[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.

Schema:
(Movie)-[IN_GENRE]->(Genre)
(Person/Director/Actor)-[ACTED_IN]->(Movie)
(Person/Director/Actor)-[DIRECTED]->(Movie)
(User)-[RATED]->(Movie)

Context:
"In a captivating mix of drama and mystery, filmmaker Gregg Araki left his mark with movies like "Totally F***ed Up," "White Bird in a Blizzard," and "Kaboom." "Totally F***ed Up," a poignant drama, featured talented actors such as James Duval, Susan Behshid, Roko Belic, and Jenee Gill in significant roles. Araki also delved into the enigmatic world of "White Bird in a Blizzard," where Shailene Woodley showcased her acting prowess. Additionally, the versatile Thomas Dekker graced the screen in "Kaboom," a film that garnered high praise from movie critic John Marshall, who rated both "Kaboom" and "Totally F***ed Up" with enthusiasm. Araki's unique storytelling and the stellar

In [ ]:
class_diz_count = {"legis_graph": 0, "recommendations": 0, "recipes": 0, "listings": 0}
examples_diz = {"legis_graph": [], "recommendations": [], "recipes": [], "listings": []}

for train_el in data["train"]:
    # Classify element
    if "DEALS_WITH" in train_el["0"]:
        classification = "legis_graph"
    elif "ACTED_IN" in train_el["0"]:
        classification = "recommendations"
    elif "COLLECTION" in train_el["0"]:
        classification = "recipes"
    elif "IN_NEIGHBORHOOD" in train_el["0"]:
        classification = "listings"
    if class_diz_count[classification] == 5:
        continue
    class_diz_count[classification] += 1
    example = "Context:\n"
    example += train_el["0"].split("Context:\n")[-1][:-4]
    example = example.replace("[/INST]", "")
    examples_diz[classification].append(example)

In [ ]:
examples_diz_text = {}
for key in examples_diz.keys():
    text = ""
    for el in examples_diz[key]:
        text += el
        text += "\n-----------------\n"
    examples_diz_text[key] = text.strip()

In [ ]:
print(examples_diz_text["legis_graph"])

Context:
"Bills like the hr5833-114, hr4036-114, hr6311-114, s1910-114, hr2864-114, hr2910-114, and s2659-114 all deal with environmental regulatory procedures."

Extracted Triples:
(hr5833-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr4036-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr6311-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(s1910-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr2864-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr2910-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(s2659-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
-----------------
Context:
"Hey guys, and welcome back to the "Decoding Bills with Knowledge Graphs" channel. Today, we're going to dive into the fascinating world of bills dealing with Members of Congress.
Now, you might be wondering, what exactly do these bills entail? Well, that's where our trusty Knowledge Graph comes in. It tells 

## Inference

In [ ]:
for el in tqdm(formatted_test_df):
    # Classify element
    if "DEALS_WITH" in el["text"]:
        classification = "legis_graph"
    elif "ACTED_IN" in el["text"]:
        classification = "recommendations"
    elif "COLLECTION" in el["text"]:
        classification = "recipes"
    elif "IN_NEIGHBORHOOD" in el["text"]:
        classification = "listings"
    text = el["text"].split("Context:\n")
    new_text = text[0] + "Here are some examples:\n" + examples_diz_text[classification] + "\nContext:\n" + text[-1]
    el["FS_prompt"] = new_text

100%|██████████| 400/400 [00:00<00:00, 173193.10it/s]


In [ ]:
print(formatted_test_df[0]["FS_prompt"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)

Here are some examples:
Context:
"Bills like the hr5833-114, hr4036-114, hr6311-114, s1910-114, hr2864-114, hr2910-114, and s2659-114 all deal with environmental regulatory procedures."

Extracted Triples:
(hr5833-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr4036-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(hr6311-114) - [DEALS_WITH] -> (Environmental regulatory procedures)
(s1910-114) - [DEALS_WITH] -> (Environmental reg

In [ ]:
# Higher model_max_length
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding_side="left")

base_model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
for el in tqdm(formatted_test_df):
    response = generate_response(el["FS_prompt"].strip())
    el["FS_triples"] = response

100%|██████████| 400/400 [1:08:04<00:00, 10.21s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FS_triples"] = el["FS_triples"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 400/400 [00:00<00:00, 317509.77it/s]


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('FSresponses.json', 'w') as f:
    json.dump(json_str, f)

# Generate responses (finetuned model)
Generate the triples starting from the test set for Mistral 7B Instruct Finetuned

In [ ]:
# importing the module
import json

# Opening JSON file
with open('FSresponses.json') as json_file:
    data = json.load(json_file)

In [ ]:
formatted_test_df = json.loads(data)

In [ ]:
print(formatted_test_df[0]["text"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)

Context:
"Hi and welcome to our show about very interesting bills.
Today's bill is the hr1818-114 bill. This bill was proposed by Ken Calvert, a Republican representative from California, and Grace Napolitano, a representative from California who is elected to the House. These two representatives are from the same state and share the sponsorship of the same bill."[/INST]

Extracted Triples:


In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Prova1/checkpoint-1000")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left")

ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    response = generate_response(el["text"].strip())
    el["FT_triples"] = response

100%|██████████| 400/400 [1:49:11<00:00, 16.38s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FT_triples"] = el["FT_triples"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 400/400 [00:00<00:00, 317509.77it/s]


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('responses.json', 'w') as f:
    json.dump(json_str, f)

# External dbs test set
Now we are going to repeat the same exact pipeline for the test_external_dbs.csv test set

In [ ]:
!pip install matplotlib
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 263.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 317.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 103.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=False,
    add_eos_token=False,
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Load Test Data

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
test_df = pd.read_csv("test_external_dbs.csv")["0"]

In [ ]:
test_df = list(test_df)

In [ ]:
formatted_test_df = []

In [ ]:
sentence = """Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)" """
sentence = sentence.strip()

In [ ]:
test_df[0]

'<s>[INST]Imagine being a Knowledge Graph constructor from unstructured text.\nFollowing the schema provided, extract all the triples you can find in the text.\n\nSchema:\n(Team)-[PARTICIPATED_IN]->(Tournament)\n(Person)-[PLAYED_IN]->(Tournament)\n(Person)-[REPRESENTS]->(Team)\n\nContext:\n""Did you know that several countries and teams have participated in various international tournaments? In France 2019, we saw Valentina Giacinti playing for her country, as well as Argentina and Japan participating. Additionally, China PR also took part in this tournament. But did you know that Argentina has also participated in USA 2003? And so did Japan! It\'s amazing how these countries and teams come together to showcase their skills and compete against each other.""[/INST]\n\nExtracted Triples:\n(Valentina Giacinti) - [PLAYED_IN] -> (France 2019)\n(Argentina) - [PARTICIPATED_IN] -> (France 2019)\n(Argentina) - [PARTICIPATED_IN] -> (USA 2003)\n(Japan) - [PARTICIPATED_IN] -> (France 2019)\n(Japan

In [ ]:
for el in tqdm(test_df):
    new_diz = {}
    splitted_el = el.split("[/INST]\n\nExtracted Triples:")
    text = splitted_el[0][3:].strip()
    text = text.split("Following the schema provided, extract all the triples you can find in the text.")
    text = sentence.join(text)
    text += "[/INST]\n\nExtracted Triples:"
    new_diz["text"] = text
    new_diz["triples"] = splitted_el[1][:-4].strip()
    formatted_test_df.append(new_diz)

100%|██████████| 210/210 [00:00<00:00, 70441.77it/s]


In [ ]:
formatted_test_df[0]

{'text': '[INST]Imagine being a Knowledge Graph constructor from unstructured text.\nFollowing the schema provided, extract all the triples you can find in the text.\nMake sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n\nSchema:\n(Team)-[PARTICIPATED_IN]->(Tournament)\n(Person)-[PLAYED_IN]->(Tournament)\n(Person)-[REPRESENTS]->(Team)\n\nContext:\n""Did you know that several countries and teams have participated in various international tournaments? In France 2019, we saw Valentina Giacinti playing for her country, as well as Argentina and Japan participating. Additionally, China PR also took part in this tournament. But did you know that Argentina has also participated in USA 2003? And so did Japan! It\'s amazing how these countries and teams come together to showcase their skills and compete against each other.""[/INST]\n\nExtracted Triples:',
 'triples': '(Valentina Giacinti) - [PLAYED_IN] -> (France 2019)\n(Argentina) - [PARTICIPATED_IN] -> (France 

# Generate responses (base model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left")

base_model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    response = generate_response(el["text"].strip())
    el["NF_triples"] = response

100%|██████████| 210/210 [43:34<00:00, 12.45s/it]


In [ ]:
print(formatted_test_df[2]["NF_triples"])

<s> [INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Team)-[PARTICIPATED_IN]->(Tournament)
(Person)-[PLAYED_IN]->(Tournament)
(Person)-[REPRESENTS]->(Team)

Context:
""Sophie Schmidt, a Canadian soccer player, played for Ghana in the China 2007 tournament. She was part of the Canada team that also participated in the same tournament. The China PR and Brazil teams were also present at the event.""[/INST]

Extracted Triples:

1. (Sophie Schmidt)-[PLAYED\_IN]->(China 2007 tournament)
2. (Canada team)-[PARTICIPATED\_IN]->(China 2007 tournament)
3. (Ghana team)-[PARTICIPATED\_IN]->(China 2007 tournament)
4. (Brazil team)-[PARTICIPATED\_IN]->(China 2007 tournament)</s>


In [ ]:
len(formatted_test_df)

210

In [ ]:
formatted_test_df[105]

{'text': '[INST]Imagine being a Knowledge Graph constructor from unstructured text.\nFollowing the schema provided, extract all the triples you can find in the text.\nMake sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n\nSchema:\n(House)-[ATTACKER, DEFENDER]->(Battle)\n(House)-[ATTACKER, DEFENDER]->(Battle)\n(Battle, Location)-[IS_IN]->(Location, Region)\n(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)\n(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)\n(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)\n(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)\n(Person,Knight, King)-[BELONGS_TO]->(House)\n(Person,Knight, King)-[HAS_STATUS]->(Status)\n(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)\n(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)\n(Person, King, Knight)-[MEMBER_OF_CULTURE]->(Culture)\n(Person, Knight, King)-[INTERACTS]->(Person, Knight, King)\n(Person, Knight, King)<-[RELATED_TO]->(Person, Knight, King)\n\nCo

In [ ]:
# We save 10 examples for the Few Shot prompting
examples= {"graph-data-science": formatted_test_df[-5:], "wwc2019": formatted_test_df[:5]}

In [ ]:
# We mantain in the list the other 100 examples
formatted_test_df = formatted_test_df[5:-5]

In [ ]:
len(formatted_test_df)

200

In [ ]:
formatted_test_df2 = formatted_test_df.copy()

In [ ]:
for el in formatted_test_df:
    el["NF_triples"] = el["NF_triples"].split("Extracted Triples:")[-1].strip()

In [ ]:
print(formatted_test_df[2]["NF_triples"])

* (Gudrun Gottschlich)-[PLAYED_IN]->(China PR)
* (Japan)-[PARTICIPATED_IN]->(Germany)
* (Japan)-[PARTICIPATED_IN]->(France)
* (Japan)-[PARTICIPATED_IN]->(China PR)
* (Japan)-[PARTICIPATED_IN]->(USA)
* (Japan)-[PARTICIPATED_IN]->(Sweden)
* (Japan)-[PARTICIPATED_IN]->(Canada)</s>


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('NFresponses.json', 'w') as f:
    json.dump(json_str, f)

# Generate responses (few shot prompting)

In [ ]:
# importing the module
import json

# Opening JSON file
with open('NFresponses.json') as json_file:
    data = json.load(json_file)

In [ ]:
formatted_test_df = json.loads(data)

In [ ]:
print(formatted_test_df[0]["text"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Team)-[PARTICIPATED_IN]->(Tournament)
(Person)-[PLAYED_IN]->(Tournament)
(Person)-[REPRESENTS]->(Team)

Context:
"France 2019 was a soccer tournament held in France, which saw participation from various teams around the world. The China PR team, represented by Li Ying, participated in the tournament. Additionally, Argentina, Australia, Brazil, Cameroon, Canada, and Chile also took part in the event."[/INST]

Extracted Triples:


## Build examples

In [ ]:
examples_diz_text = {}
for key in examples.keys():
    text = ""
    for el in examples[key]:
        text += "Context:\n" + el["text"].split("Context:\n")[-1] + "\n" + el["triples"]
        text += "\n-----------------\n"
    examples_diz_text[key] = text.replace("[INST]", "").replace("[/INST]", "").strip()

In [ ]:
print(examples_diz_text["graph-data-science"])

Context:
""Feast for Crows" is a novel written by George R.R Martin and published in 2013. The book is set in the fictional world of Westeros and follows the events of the Red Wedding, which marked the end of the War of the Five Kings. In this novel, several characters appear, including Robin, Meg, Jeyne Darry, Marianne Vance, Malwyn Frey, and Raynard. These characters are all present at the Feast for Crows, where they witness important events and interact with each other."

Extracted Triples:
(Robin) - [APPEARED_IN] -> (Feast for crows)
(Meg) - [APPEARED_IN] -> (Feast for crows)
(Jeyne Darry) - [APPEARED_IN] -> (Feast for crows)
(Marianne Vance) - [APPEARED_IN] -> (Feast for crows)
(Malwyn Frey) - [APPEARED_IN] -> (Feast for crows)
(Raynard) - [APPEARED_IN] -> (Feast for crows)
-----------------
Context:
""Welcome back to our channel where we dive into the world of Game of Thrones and explore its intricate web of characters and events. Today, we'll take a closer look at some key figur

## Inference

In [ ]:
for el in tqdm(formatted_test_df):
    # Classify element
    if "DEFENDER_KING" in el["text"]:
        classification = "graph-data-science"
    elif "PLAYED_IN" in el["text"]:
        classification = "wwc2019"
    text = el["text"].split("Context:\n")
    new_text = text[0] + "Here are some examples:\n" + examples_diz_text[classification] + "\nContext:\n" + text[-1]
    el["FS_prompt"] = new_text

100%|██████████| 200/200 [00:00<00:00, 198499.95it/s]


In [ ]:
print(formatted_test_df[0]["FS_prompt"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Team)-[PARTICIPATED_IN]->(Tournament)
(Person)-[PLAYED_IN]->(Tournament)
(Person)-[REPRESENTS]->(Team)

Here are some examples:
Context:
""Did you know that several countries and teams have participated in various international tournaments? In France 2019, we saw Valentina Giacinti playing for her country, as well as Argentina and Japan participating. Additionally, China PR also took part in this tournament. But did you know that Argentina has also participated in USA 2003? And so did Japan! It's amazing how these countries and teams come together to showcase their skills and compete against each other.""

Extracted Triples:
(Valentina Giacinti) - [PLAYED_IN] -> (France 2019)
(Argentina) - [PARTICIPATED_IN] -> (France 2019)
(Argentina) - [P

In [ ]:
# Higher model_max_length
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding_side="left")

base_model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
for el in tqdm(formatted_test_df):
    response = generate_response(el["FS_prompt"].strip())
    el["FS_triples"] = response

100%|██████████| 200/200 [28:38<00:00,  8.59s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FS_triples"] = el["FS_triples"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 126965.46it/s]


In [ ]:
print(formatted_test_df[0]["FS_triples"])

(Li Ying) - [PLAYED_IN] -> (France 2019)
(China PR) - [PARTICIPATED_IN] -> (France 2019)
(Argentina) - [PARTICIPATED_IN] -> (France 2019)
(Australia) - [PARTICIPATED_IN] -> (France 2019)
(Brazil) - [PARTICIPATED_IN] -> (France 2019)
(Cameroon) - [PARTICIPATED_IN] -> (France 2019)
(Canada) - [PARTICIPATED_IN] -> (France 2019)
(Chile) - [PARTICIPATED_IN] -> (France 2019)</s>


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('FSresponses2.json', 'w') as f:
    json.dump(json_str, f)

# Generate responses (finetuned model)

In [ ]:
# importing the module
import json

# Opening JSON file
with open('FSresponses.json') as json_file:
    data = json.load(json_file)

In [ ]:
formatted_test_df = json.loads(data)

In [ ]:
print(formatted_test_df[0]["text"])

[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.
Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"

Schema:
(Team)-[PARTICIPATED_IN]->(Tournament)
(Person)-[PLAYED_IN]->(Tournament)
(Person)-[REPRESENTS]->(Team)

Context:
"France 2019 was a soccer tournament held in France, which saw participation from various teams around the world. The China PR team, represented by Li Ying, participated in the tournament. Additionally, Argentina, Australia, Brazil, Cameroon, Canada, and Chile also took part in the event."[/INST]

Extracted Triples:


In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Prova1/checkpoint-1000")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left")

ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    text = el["text"].replace('Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n', '')
    response = generate_response(text.strip())
    el["FT_triples"] = response

100%|██████████| 200/200 [47:36<00:00, 14.28s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FT_triples"] = el["FT_triples"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 199254.35it/s]


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('responses_external_dbs.json', 'w') as f:
    json.dump(json_str, f)

# First Fine-tuned model with Few Shot prompting
We use the first fine-tuned model (the one finetuned without examples in its observations). We give it in input the same observations we used for the Base Model with Few Shot prompting.

In [ ]:
for el in tqdm(formatted_test_df):
    text = el["FS_prompt"].replace('Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n', '')
    response = generate_response(text.strip())
    el["FT_FS_triples"] = response

100%|██████████| 200/200 [51:55<00:00, 15.58s/it] 


In [ ]:
for el in tqdm(formatted_test_df):
    el["FT_FS_triples"] = el["FT_FS_triples"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 107808.87it/s]


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('responses_external_dbs.json', 'w') as f:
    json.dump(json_str, f)

# Second Fine-tuned model
Now we use the second fine-tuned model (the one finetuned WITH examples in its observations).

## Checkpoint 50

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Prova_FS/checkpoint-50")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding_side="left")

ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    text = el["FS_prompt"].replace('Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n', '')
    response = generate_response(text.strip())
    el["FSFT_triples_50"] = response

100%|██████████| 200/200 [1:12:13<00:00, 21.67s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FSFT_triples_50"] = el["FSFT_triples_50"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 124367.80it/s]


## Checkpoint 150

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Prova_FS/checkpoint-150")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding_side="left")

ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    text = el["FS_prompt"].replace('Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n', '')
    response = generate_response(text.strip())
    el["FSFT_triples_150"] = response

100%|██████████| 200/200 [1:01:40<00:00, 18.50s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FSFT_triples_150"] = el["FSFT_triples_150"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 124367.80it/s]


## Checkpoint 300

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Prova_FS/checkpoint-300")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding_side="left")

ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def generate_response(eval_prompt):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512, repetition_penalty=1.15, pad_token_id=tokenizer.unk_token_id)[0], skip_special_tokens=False)
    return response

In [ ]:
for el in tqdm(formatted_test_df):
    text = el["FS_prompt"].replace('Make sure to mantain the formatting of the schema: "(subject)-[PREDICATE]->(Object)"\n', '')
    response = generate_response(text.strip())
    el["FSFT_triples_300"] = response

100%|██████████| 200/200 [1:03:53<00:00, 19.17s/it]


In [ ]:
for el in tqdm(formatted_test_df):
    el["FSFT_triples_300"] = el["FSFT_triples_300"].split("Extracted Triples:\n")[-1].strip()

100%|██████████| 200/200 [00:00<00:00, 34262.99it/s]


In [ ]:
import json
json_str = json.dumps(formatted_test_df)

In [ ]:
with open('responses_external_dbs.json', 'w') as f:
    json.dump(json_str, f)